In [ ]:
from exasol.nb_connector.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for other supporting NBs from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/popup_message_ui.ipynb')}

In [ ]:
from typing import Callable, List, Tuple
from tempfile import TemporaryFile
from contextlib import ExitStack
from datetime import datetime
from dateutil.relativedelta import relativedelta
import io

import ipywidgets as widgets
import requests
from exasol.nb_connector.secret_store import Secrets
from exasol.nb_connector.connections import open_pyexasol_connection


def _transform_flight_row(row: str) -> str:

    fields = row.split(',')
    fields[0] = datetime.strptime(fields[0], '%m/%d/%Y %I:%M:%S %p').strftime('%Y-%m-%d')
    for i in [10, 12]:
        fields[i] = 'False' if float(fields[i]) == 0. else 'True'
    return ','.join(fields)


def _transform_airline_row(row: str) -> str:

    fields = row.split(',', maxsplit=1)
    al_name = fields[1].strip('"').split(':')[0]
    fields[1] = f'"{al_name}"'
    return ','.join(fields) + '\n'


def _import_from_cloudfront(conf: Secrets, file_name: str, 
                            transform: Callable[[str], str], table_name: str) -> None:

    # Read requested csv file from the cloudfront
    response = requests.get(f'https://d1je7p5oh8pade.cloudfront.net/{file_name}')
    response.raise_for_status()
    content_stream = io.BytesIO(response.content)
    f_src = io.TextIOWrapper(content_stream, encoding='utf-8')

    with ExitStack() as stack:
        f_tmp = stack.enter_context(TemporaryFile())

        # Transform the source file
        line_count = 0
        while True:
            src_line = f_src.readline()
            if not src_line:
                break
            elif line_count > 0:
                tx_line = transform(src_line)
                f_tmp.write(bytes(tx_line, 'utf-8'))
            line_count += 1
        f_tmp.seek(0)

        # Import the transformed file
        pyexasol_conn = stack.enter_context(open_pyexasol_connection(
            conf, schema=conf.db_schema, compression=True))
        import_params = {
            'ROW SEPARATOR': 'LF',
            'COLUMN SEPARATOR': ','
        }
        pyexasol_conn.import_from_file(f_tmp, table=table_name, import_params=import_params)


def load_flights_data(conf: Secrets, months: List[str]) -> None:

    for mon in months:
        file_name = f'US_FLIGHTS_{mon.replace(" ", "_").upper()}.csv'
        _import_from_cloudfront(conf, file_name, _transform_flight_row, 'US_FLIGHTS')


def load_airlines_data(conf: Secrets) -> None:

    _import_from_cloudfront(conf, 'US_AIRLINES.csv', _transform_airline_row, 'US_AIRLINES')


def get_data_selection_ui(conf: Secrets) -> widgets.Widget:
    """
    Builds a UI with a multi-select list of data periods and buttons for
    loading the selected data.
    """

    ui_look = get_config_styles()

    start_date = datetime(year=2023, month=4, day=1)
    months = [(start_date + relativedelta(months=i)).strftime('%b %Y')
              for i in range(12)]

    data_selector = widgets.SelectMultiple(options=months, layout=ui_look.input_layout, style=ui_look.input_style)
    flights_btn = widgets.Button(description='Load Flights', style=ui_look.button_style, layout=ui_look.button_layout)
    airlines_btn = widgets.Button(description='Load Airlines', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Data Periods', style=ui_look.header_style, layout=ui_look.header_layout)

    def load_flights(btn):
        if data_selector.value:
            try:
                load_flights_data(conf, data_selector.value)
                popup_message('Flights data has been loaded successfully')
                data_selector.options = [opt for opt in data_selector.options 
                                         if opt not in data_selector.value]
                btn.icon = 'check'
                btn.disabled = not data_selector.options
            except Exception as ex:
                popup_message(f'Failed to load the flights data: {ex}')
        else:
            btn.icon = 'check'

    def load_airlines(btn):
        try:
            load_airlines_data(conf)
            popup_message('Airlines data has been loaded successfully')
            btn.disabled = True
        except Exception as ex:
            popup_message(f'Failed to load the airlines data: {ex}')

    def on_value_change(change):
        flights_btn.icon = 'pen'

    flights_btn.on_click(load_flights)
    airlines_btn.on_click(load_airlines)
    data_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([data_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), 
             widgets.Box([flights_btn, airlines_btn], layout=ui_look.row_layout)]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui